In [59]:
import math
x = [1, 2]
y = [1, -6]


math.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2) # Euclidean distance

8.0

In [60]:
(x[0]*y[0] + x[1]*y[1]) / (4*6)   # Dot product

-0.4583333333333333

In [61]:
import torch 

In [62]:
sentence = "this is a sample sentence for embedding"
sentence2 = "this is sentence embedding"

dc = {s:i for i,s in enumerate(sorted(sentence.replace(',', '').split()))}
dc

{'a': 0,
 'embedding': 1,
 'for': 2,
 'is': 3,
 'sample': 4,
 'sentence': 5,
 'this': 6}

In [63]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=10,binary=True)
cv.fit([sentence])
cv.transform([sentence]).todense()

matrix([[1, 1, 1, 1, 1, 1]])

In [64]:
cv.transform([sentence2]).todense()
cv.transform(["this"]).todense()

matrix([[0, 0, 0, 0, 0, 1]])

In [65]:
sentence_int = torch.tensor( [dc[s] for s in sentence.replace(',', '').split()])
print(sentence_int) # Tokenized sequence

tensor([6, 3, 0, 4, 5, 2, 1])


In [66]:
len(dc)  # Vocabulary size

7

In [67]:
vocab_size = len(dc) # 768 - 4096
emb = torch.nn.Embedding(vocab_size, 3) # Zufallsinitialisierte Embedding-Matrix
emb.weight.data # token 0 

tensor([[ 0.8707,  1.1182, -0.6511],
        [-1.8573,  1.5171, -1.4076],
        [ 0.3429,  0.6065,  1.7809],
        [ 0.6241, -1.6505,  0.3585],
        [ 0.5940,  1.0827, -1.6407],
        [ 0.5672,  0.5458,  0.1611],
        [-1.0457,  1.0252,  0.2374]])

In [68]:
sentence_emb = emb(sentence_int).detach() # Embedding-Vektoren für den Satz
sentence_emb

tensor([[-1.0457,  1.0252,  0.2374],
        [ 0.6241, -1.6505,  0.3585],
        [ 0.8707,  1.1182, -0.6511],
        [ 0.5940,  1.0827, -1.6407],
        [ 0.5672,  0.5458,  0.1611],
        [ 0.3429,  0.6065,  1.7809],
        [-1.8573,  1.5171, -1.4076]])

In [90]:
# Q, K , V

d = sentence_emb.shape[1]  
d_Q, d_K, d_V = 2, 2, 4 # Dimensionen für Q, K, V

W_Q_L = torch.nn.Linear(d_Q, d, bias=False) 
# oder 
W_Q = torch.nn.Parameter(torch.rand(d, d_Q)) 
#W_Q.weight.data
W_K = torch.nn.Parameter(torch.rand(d, d_K))
W_V = torch.nn.Parameter(torch.rand(d, d_V))

W_K


Parameter containing:
tensor([[0.7119, 0.7715],
        [0.0393, 0.1200],
        [0.2610, 0.1779]], requires_grad=True)

In [85]:
x_0 = sentence_emb[0] # Erster Token aus Embedding-Satz
q_0 = x_0 @ W_Q # drei Repräsentationen    
k_0 = x_0 @ W_K   
v_0 = x_0 @ W_V   
q_0, k_0, v_0

(tensor([-0.2080, -0.1308], grad_fn=<SqueezeBackward4>),
 tensor([0.1106, 0.4931], grad_fn=<SqueezeBackward4>),
 tensor([-0.1909, -0.4126,  0.0817,  0.7000], grad_fn=<SqueezeBackward4>))

In [ ]:
keys = sentence_emb @ W_K  # Alle Keys
values = sentence_emb @ W_V  # Alle Values
queries = sentence_emb @ W_Q  # Alle Queries

scores_raw = queries @ keys.T # QK^T für alle Tokens / Relevanzmaß


tensor([[-0.0875,  0.1129, -0.1526,  0.1090, -0.1993, -0.5406,  0.3016],
        [-0.3419,  0.6698, -0.8075, -0.2578, -0.6523, -1.2791,  0.3563],
        [ 0.8805, -1.5281,  1.8977,  0.0751,  1.7889,  4.0116, -1.6254],
        [ 0.5974, -1.1973,  1.4357,  0.5306,  1.1250,  2.1374, -0.5261],
        [ 0.5638, -0.9002,  1.1429, -0.1853,  1.1885,  2.8529, -1.3213],
        [ 0.6631, -0.8664,  1.1666, -0.7927,  1.5041,  4.0556, -2.2449],
        [-0.5071,  0.5530, -0.7910,  0.9333, -1.2106, -3.4998,  2.1099]],
       grad_fn=<MmBackward0>)

In [88]:
import torch.nn.functional as F

scores = scores_raw / math.sqrt(d_K)  # Skalierung
attn_weights = F.softmax(scores, dim=1)  # Softmax über Zeilen
attn_weights  # Aufmerksamkeitsgewichte

tensor([[0.1384, 0.1595, 0.1322, 0.1591, 0.1279, 0.1005, 0.1823],
        [0.1285, 0.2628, 0.0925, 0.1364, 0.1032, 0.0662, 0.2105],
        [0.0666, 0.0121, 0.1366, 0.0377, 0.1265, 0.6092, 0.0113],
        [0.1121, 0.0315, 0.2028, 0.1069, 0.1628, 0.3331, 0.0507],
        [0.0969, 0.0344, 0.1460, 0.0571, 0.1508, 0.4892, 0.0256],
        [0.0622, 0.0211, 0.0888, 0.0222, 0.1127, 0.6850, 0.0080],
        [0.0725, 0.1534, 0.0593, 0.2007, 0.0441, 0.0087, 0.4613]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
# Attention(Q,K,V) = softmax(QK^T/sqrt(d_K))V
attn_output = attn_weights @ values  # Gewichtetet Summe der Values
attn_output  # Kontextvektor 

tensor([[-0.0725, -0.0094, -0.0852,  0.2169],
        [-0.1551, -0.0961, -0.2000,  0.0060],
        [ 0.8433,  0.6507,  1.2213,  1.3927],
        [ 0.4499,  0.4126,  0.6285,  0.9006],
        [ 0.6777,  0.5400,  0.9773,  1.1722],
        [ 0.9284,  0.6853,  1.3614,  1.4959],
        [-0.6461, -0.5189, -0.7877, -0.2367]], grad_fn=<MmBackward0>)